In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
rust_gpp_url = 'https://benchmarksgame-team.pages.debian.net/benchmarksgame/fastest/rust-gpp.html'
with urlopen(rust_gpp_url) as f:
    # lxmlでないと閉じタグを省略しているbenchmarksgameのHTMLをパースできない
    soup = BeautifulSoup(f, 'lxml')

In [3]:
table = {'Rust': {}, 'C++': {}}
for key_values in table.values():
    key_values['secs'] = []
    key_values['mem'] = []
    key_values['gz'] = []
    key_values['busy'] = []

# 問題はtbody毎に分かれている
tbodies = soup.find_all('tbody')
# 最後の表は処理系の情報なので取り除く
tbodies = tbodies[:-1]
for tbody in tbodies:
    trs = tbody.find_all('tr')
    # 3, 4番目のtr要素がそれぞれRust, C++のベンチマークである
    for table_key_values, values in zip(table.values(), trs[2:4]):
        # その中のtd要素の2-5番目がそれぞれsecs, mem, gz, busyである
        for table_values, value in zip(table_key_values.values(), values.find_all('td')[1:5]):
            # memの値にコンマが含まれているので取り除く
            value = value.string.replace(',', '')
            table_values.append(float(value))

print(table)

{'Rust': {'secs': [1.69, 6.07, 3.32, 1.75, 10.42, 1.97, 1.51, 1.7, 2.13, 5.17], 'mem': [994656.0, 876.0, 199724.0, 3060.0, 876.0, 2220.0, 2036.0, 32308.0, 153508.0, 135372.0], 'gz': [1330.0, 1753.0, 721.0, 1366.0, 1016.0, 1126.0, 1906.0, 1332.0, 986.0, 1841.0], 'busy': [2.96, 6.18, 12.07, 1.83, 40.8, 7.88, 4.59, 6.69, 3.18, 15.76]}, 'C++': {'secs': [4.72, 7.7, 3.93, 1.89, 10.7, 1.98, 1.46, 1.51, 1.85, 3.89], 'mem': [500116.0, 1772.0, 113768.0, 4460.0, 1864.0, 2320.0, 2180.0, 25708.0, 203748.0, 156148.0], 'gz': [840.0, 1879.0, 809.0, 513.0, 980.0, 1044.0, 2711.0, 1791.0, 1315.0, 1631.0], 'busy': [4.76, 8.02, 12.78, 1.95, 42.31, 7.9, 4.39, 6.01, 4.44, 11.99]}}


In [4]:
print('C++ / Rust')

for (key, rust_values), cpp_values in zip(table['Rust'].items(), table['C++'].values()):
    n = len(rust_values)
    # 中央値の計算のために収集する
    ratios = []
    mean = 1.0
    for rust_value, cpp_value in zip(rust_values, cpp_values):
        ratio = cpp_value / rust_value
        ratios.append(ratio)
        mean *= ratio
    mean **= 1 / n
    print(f'{key.ljust(4)} - mean: {mean:.2f}', end=', ')

    ratios.sort()
    if n % 2 == 1:
        median = ratios[n // 2]
    else:
        # 中央値が2つある場合はそれらの相乗平均を取る
        median = (ratios[n // 2 - 1] * ratios[n // 2]) ** (1 / 2)
    print(f'median: {median:.2f}')

C++ / Rust
secs - mean: 1.10, median: 1.02
mem  - mean: 1.09, median: 1.11
gz   - mean: 0.95, median: 1.02
busy - mean: 1.08, median: 1.05
